In [31]:
import collections
def main():
    
    """
    1. For every party ID caclulate number of CTU's imputed
    2. Calculate number of distinct CTU's per party id
    3. Devide number of CTU's imputed by distinct number of CTUs
    4. Create buckets 0.00% -24%, 25% - 49%, 50 - 74%, 75% - 98%, 99% -100 
    in every bucket all ther partyid those that fit inside thier backet
    5. Caclulcate proportion of partyids that fit inside the bucket out of total partyids
        Calculate proportion of accounts that have more than:	
        99% missing	0.20
        75% missing	0.40
        50% missing	0.60
        25% missing	0.70

    """
    
    imputed_df = get_imputed_df()
    #imputed_df = get_test_df()
    num_imputed_ctus_per_partyid = get_num_imputed_ctus_per_partyid(imputed_df)
    num_distinct_ctus_per_partyid = get_num_distinct_ctus_per_partyid(imputed_df)
    joined_num_distinct_imputed_ctus_df = join(num_distinct_ctus_per_partyid,\
                                               num_imputed_ctus_per_partyid)
    percentage_of_missing_ctus_per_partyid = \
                get_percentage_of_missing_ctus_per_party_id( 
                        joined_num_distinct_imputed_ctus_df )

    party_id_and_its_bucket = create_buckets( 
                                        percentage_of_missing_ctus_per_partyid)
    num_partyids_with_missing_ctus_per_backet = get_num_partyids_per_backet( 
                                          party_id_and_its_bucket )
    total_num_ids = imputed_df.groupby("party_id").count().count()
    calculate_proportion_of_missing_ctus_per_percentile ( num_partyids_with_missing_ctus_per_backet, \
                                          total_num_ids )
    


In [32]:
buckets_df = main()

 proportion of partys with more than  0%  missing ctus  0.05
 proportion of partys with more than  25%  missing ctus  0
 proportion of partys with more than  50%  missing ctus  0
 proportion of partys with more than  75%  missing ctus  0
 proportion of partys with more than  99%  missing ctus  0


In [25]:

def test_create_buckets():
        """
        +--------+-----------------------+-------+
        |party_id|percentage_missing_ctus|buckets|
        +--------+-----------------------+-------+
        |       1|                    0.2|    0.0|
        |       2|                   0.33|    1.0|
        |       3|                    1.0|    4.0|
        |       4|                   0.75|    3.0|
        |       5|                    0.6|    2.0|
        |       6|                    0.6|    2.0|
        +--------+-----------------------+-------+
        we are testing if for party id 6  we get the bucket number 2
        """
        spark = start_spark_session()
        percentage_of_missing_ctus_per_partyid = spark.createDataFrame([
                       (1,0.2), 
                       (2,0.33), 
                       (3,1.0), 
                       (4,0.75), 
                       (5,0.6), 
                       (6,0.6), 
                      ],['party_id','percentage_missing_ctus'] )      
        bucketized_percentages_of_missing_ctu =create_buckets(
                        percentage_of_missing_ctus_per_partyid)
        bucket = bucketized_percentages_of_missing_ctu.where("party_id = 6")\
                    .select("buckets").collect()[0] # finding out that party id 6 is in the bucket 2
        bucket_number = 2 # bucket number fot the party_id 6
        if bucket[0] == bucket_number:
            print("Test passed")

In [26]:
def test_get_num_partyids_per_backet():
    spark = start_spark_session()
    party_id_and_its_bucket = spark.createDataFrame([
        (1,0),
        (2,1),
        (3,4),
        (4,3),
        (5,2),
        (6,2)
    ], ['party_id', 'buckets'])
    num_partyids_per_backet = get_num_partyids_per_backet( party_id_and_its_bucket )
    test_sample_of_num_partyids_per_backet = [ [4,1], [3,1], [2,2], [1,1], [0,1]]
    num_partyids_per_backet_list_of_lists = [list(x) for x in num_partyids_per_backet] # makes it list of list instead of list of pyspark rows
    if test_sample_of_num_partyids_per_backet == num_partyids_per_backet_list_of_lists:
        print("Test passed")


In [27]:
test_create_buckets()
test_get_num_partyids_per_backet()

Test passed
Test passed


In [28]:
"""
Total number of rows imputed for an account  / 
devided   by total number of  distinct  CTUs for that customer
"""

from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import Bucketizer
from pyspark.sql.functions import countDistinct


def start_spark_session():
    """
    Starting spark session
    """

    spark = SparkSession \
        .builder \
        .appName("Python Spark SQL basic example") \
        .config("spark.some.config.option", "some-value") \
        .getOrCreate()

    return spark

def get_test_df():
    """
    Creating test data frame 
    """
    spark = start_spark_session()
    
    return spark.createDataFrame([
                       (1,1,0), (1,2,0), (1,3,0), (1,4,1), (1,5,0),
                       (2,1,0), (2,2,0), (2,3,1),
                       (3,1,1), (3,2,1), (3,3,1),
                       (4,1,1), (4,2,1), (4,3,1), (4,4,0),
                       (5,1,1), (5,2,1), (5,3,1), (5,4,0), (5,5,0),
                       (6,1,1), (6,2,1), (6,3,1), (6,4,0), (6,5,0),
                       (7,1,0), (7,2,0), (6,3,0), (6,4,0), (6,5,0),
                      ],['party_id','ctu','imputed_ctu'] )


def get_imputed_df():
    """
    Load rack files: genereal preprocessing, preprocessing, imputed
    """
    spark = start_spark_session()
    
    imputed_file_name = "../data/imputed_train_2020_06_30_1.csv"
    imputed_df = spark.read.format("csv").option("header", "true").load(imputed_file_name)
    
    return imputed_df

def get_num_imputed_ctus_per_partyid( imputed_df ):
    """
    return df with number of imputed ctus for each party id
    """
    num_imputed_ctus_per_partyid = imputed_df.groupby('party_id','imputed_ctu').count()\
            .where("imputed_ctu = 1")\
            .select("party_id", col("count").alias("num_imputed_ctus"))
    return num_imputed_ctus_per_partyid
    
    
def get_num_distinct_ctus_per_partyid( imputed_df ):
    """
    Calculating number of distinct
    CTUs per customer
    """
    distinct_ctus_per_partyid = imputed_df.groupby('party_id').agg(countDistinct("CTU"))\
        .select("party_id", col("count(CTU)").alias("distinct_ctus"))
    return distinct_ctus_per_partyid


def join( distinct_ctus_per_partyid, num_imputed_ctus_per_partyid ):
    """
    joining number of disinct ctus per partyid with number of imputed stus per party id
    """
    imputed_distinct_cuts_nums_for_partyid = num_imputed_ctus_per_partyid.alias('a').\
        join(distinct_ctus_per_partyid.alias('b'), 
        col('a.party_id') == col('b.party_id'))
 
    return imputed_distinct_cuts_nums_for_partyid


def get_percentage_of_missing_ctus_per_party_id( imputed_distinct_ctus_df ):
    
    percentage_of_missing_ctus = imputed_distinct_ctus_df.withColumn\
        ("percentage_missing_ctus", col('num_imputed_ctus') / col('distinct_ctus') )
    
    return percentage_of_missing_ctus

def create_buckets( percentage_of_missing_ctus_per_partyid ):
    """
    Devide party ids by percentage of missing ctus into a list of 5 buckets
    > 0   < 0.25
    > 0.25 < 0.5
    > 0.5 < 0.75
    > 0.75 < 0.99
    > 0.99
    Output:
    +--------+-----------------------+-------+
    |party_id|percentage_missing_ctus|buckets|
    +--------+-----------------------+-------+
    |       1|                    0.2|    0.0|
    |       2|                   0.33|    1.0|
    |       3|                    1.0|    4.0|
    |       4|                   0.75|    3.0|
    |       5|                    0.6|    2.0|
    |       6|                    0.6|    2.0|
    +--------+-----------------------+-------+
    """
 
    bucketizer = Bucketizer(splits=[ 0, 0.25, 0.5, 0.75, 0.99, float('Inf') ], \
                            inputCol="percentage_missing_ctus", outputCol="buckets")
    df_of_buckets_ratio_between_imputed_distinct_ctus\
            = bucketizer.setHandleInvalid("keep").\
            transform(percentage_of_missing_ctus_per_partyid)
    return df_of_buckets_ratio_between_imputed_distinct_ctus


def get_num_partyids_per_backet( buckets_df ):
    """
    returns a list of rows
    """
    rows_list = buckets_df.groupby("buckets").count().\
                    orderBy("buckets", ascending = False).collect()
    return rows_list




def calculate_proportion_of_missing_ctus_per_percentile ( num_partyids_with_missing_ctus_per_bucket , total_num_partyids ):
    """
    Caclulate total number of rows
    Start with bucket 4
    4. Proportion of partyids with more than 99% missing ctus. start iteration from bucket 4 get num of partyids with 99% or up 
    that have missing ctus devide it by total number of partyids
    3. Porportion of partyids with more 75% missing ctus . next we find out how many elements in bucket 3, 
    we get the value and then do plus bucket 4 and devive by total number of elements,
    2.  Porportion of partyids with more than 50% missing ctus
    1. Porportion of partyids with more than 25% missing ctus
    0. Porportion of partyids with more than 0% missing ctus
    """
    
    partyid_total_missing_ctus = 0 
    missing_percentage_per_bucket =  {4: '99%', 3: '75%', 2: '50%', 1: '25%', 0: '0%'}
    dict_to_sort = {}
    for bucket_num_partyids in num_partyids_with_missing_ctus_per_bucket:
        bucket = int(bucket_num_partyids[0])
        percent = missing_percentage_per_bucket.pop( bucket )
        #print(missing_percentage_per_bucket)
        party_ids_count = bucket_num_partyids[1]
        partyid_total_missing_ctus += party_ids_count
        dict_to_sort [percent] = round(partyid_total_missing_ctus / total_num_partyids,2) 
    # print the rest of the bucket where we have zero missing ctus
    for percent in missing_percentage_per_bucket.values():
        dict_to_sort [percent] = 0
    od = collections.OrderedDict(sorted(dict_to_sort.items()))  # sort dictionary by percentage
    for percent , proportion in od.items():
       print(' proportion of partys with more than ', percent, ' missing ctus ', proportion )

